## Preparacion de datos - tabla minable

In [0]:
df = spark.sql("SELECT * FROM default.candidatos_creditosss")
df.show()


+---------+-----------+-------+---------+-----------+-------------+-------------+-----------+--------------+----------+------------------+-------+-------+---------+
|client_id|district_id|loan_id|date_loan|loan_amount|loan_duration|loan_payments|loan_status|buen_candidato|account_id| account_frequency|date_ac|disp_id|     type|
+---------+-----------+-------+---------+-----------+-------------+-------------+-----------+--------------+----------+------------------+-------+-------+---------+
|        2|          1|   4959|   940105|      80952|           24|       3373.0|          A|             1|         2|  POPLATEK MESICNE| 930226|      2|    OWNER|
|        3|          1|   4959|   940105|      80952|           24|       3373.0|          A|             1|         2|  POPLATEK MESICNE| 930226|      3|DISPONENT|
|       25|         21|   4961|   960429|      30276|           12|       2523.0|          B|             0|        19|  POPLATEK MESICNE| 950407|     25|    OWNER|
|       31

In [0]:
spark

In [0]:
#visualizamos si existe nulos en la estrcutura de las tablas 
df.printSchema()

root
 |-- client_id: integer (nullable = true)
 |-- district_id: integer (nullable = true)
 |-- loan_id: integer (nullable = true)
 |-- date_loan: integer (nullable = true)
 |-- loan_amount: integer (nullable = true)
 |-- loan_duration: integer (nullable = true)
 |-- loan_payments: double (nullable = true)
 |-- loan_status: string (nullable = true)
 |-- buen_candidato: integer (nullable = true)
 |-- account_id: integer (nullable = true)
 |-- account_frequency: string (nullable = true)
 |-- date_ac: integer (nullable = true)
 |-- disp_id: integer (nullable = true)
 |-- type: string (nullable = true)



In [0]:
from pyspark.sql.functions import col, sum

# Crear una lista con el número de nulos por cada columna
df.select([sum(col(c).isNull().cast("int")).alias(c) for c in df.columns]).show()


+---------+-----------+-------+---------+-----------+-------------+-------------+-----------+--------------+----------+-----------------+-------+-------+----+
|client_id|district_id|loan_id|date_loan|loan_amount|loan_duration|loan_payments|loan_status|buen_candidato|account_id|account_frequency|date_ac|disp_id|type|
+---------+-----------+-------+---------+-----------+-------------+-------------+-----------+--------------+----------+-----------------+-------+-------+----+
|        0|          0|      0|        0|          0|            0|            0|          0|             0|         0|                0|      0|      0|   0|
+---------+-----------+-------+---------+-----------+-------------+-------------+-----------+--------------+----------+-----------------+-------+-------+----+



In [0]:
#cantidad de registros y columnas
print(f"Filas: {df.count()} | Columnas: {len(df.columns)}")


Filas: 827 | Columnas: 14


In [0]:
display(df.describe())


summary,client_id,district_id,loan_id,date_loan,loan_amount,loan_duration,loan_payments,loan_status,buen_candidato,account_id,account_frequency,date_ac,disp_id,type
count,827,827,827,827,827,827,827,827,827,827,827,827,827,827
mean,7171.419588875454,38.21523579201935,6180.746070133011,962950.1644498187,151801.53808948005,36.26118500604595,4231.251511487303,null,0.9081015719467956,5863.874244256348,null,952186.3966142684,7049.6348246674725,null
stddev,4053.288032670533,25.033480922663394,681.2927327821689,14545.976879692378,114577.04184808594,17.172800162528755,2238.757849040662,null,0.28905732959348196,3277.9255060772075,null,14143.35654642866,3925.311133350347,null
min,2,1,4959,930705,4980,12,304.0,A,0,2,POPLATEK MESICNE,930113,2,DISPONENT
max,13971,77,7308,981208,590820,60,9910.0,D,1,11362,POPLATEK TYDNE,971222,13663,OWNER


In [0]:
#eliminar los ids
df = df.drop("client_id", "district_id", "loan_id", "disp_id", "account_id" )
df.printSchema()

root
 |-- date_loan: integer (nullable = true)
 |-- loan_amount: integer (nullable = true)
 |-- loan_duration: integer (nullable = true)
 |-- loan_payments: double (nullable = true)
 |-- loan_status: string (nullable = true)
 |-- buen_candidato: integer (nullable = true)
 |-- account_frequency: string (nullable = true)
 |-- date_ac: integer (nullable = true)
 |-- type: string (nullable = true)



In [0]:
nulos_totales = df.select([col(c).isNull() for c in df.columns]).rdd.flatMap(lambda x: x).sum()

if nulos_totales > 0:
    print(f"Existen {nulos_totales} valores nulos en el DataFrame.")
else:
    print("No existen valores nulos en el DataFrame.")
#no existen datos nullos como se puede ver esta vacio

No existen valores nulos en el DataFrame.


In [0]:
from pyspark.sql.functions import from_unixtime, col

# Si los valores son días desde el Unix epoch, conviértelos:
df = df.withColumn("date_loan", from_unixtime(col("date_loan") * 86400).cast("date"))
df = df.withColumn("date_ac", from_unixtime(col("date_ac") * 86400).cast("date"))

df.select("date_loan", "date_ac").show(truncate=False)



+----------+----------+
|date_loan |date_ac   |
+----------+----------+
|1957-12-24|1930-12-07|
|1957-12-24|1930-12-07|
|2013-08-16|1986-03-09|
|1907-01-14|2014-06-11|
|1933-11-19|1905-12-20|
|1932-04-03|1905-12-10|
|2013-10-28|1960-06-25|
|1905-12-12|2013-10-31|
|1905-12-12|2013-10-31|
|1907-01-12|2013-04-19|
|1934-05-29|1905-09-03|
|1906-03-20|2014-05-31|
|2015-06-24|2013-11-06|
|2015-06-24|2013-11-06|
|1959-02-23|1933-05-25|
|1959-02-23|1933-05-25|
|1904-11-07|2014-09-03|
|1933-01-21|1904-05-04|
|1934-05-26|1907-01-14|
|1960-09-17|1933-08-09|
+----------+----------+
only showing top 20 rows



In [0]:
df.show()

+----------+-----------+-------------+-------------+-----------+--------------+------------------+----------+---------+
| date_loan|loan_amount|loan_duration|loan_payments|loan_status|buen_candidato| account_frequency|   date_ac|     type|
+----------+-----------+-------------+-------------+-----------+--------------+------------------+----------+---------+
|1957-12-24|      80952|           24|       3373.0|          A|             1|  POPLATEK MESICNE|1930-12-07|    OWNER|
|1957-12-24|      80952|           24|       3373.0|          A|             1|  POPLATEK MESICNE|1930-12-07|DISPONENT|
|2013-08-16|      30276|           12|       2523.0|          B|             0|  POPLATEK MESICNE|1986-03-09|    OWNER|
|1907-01-14|      30276|           12|       2523.0|          A|             1|  POPLATEK MESICNE|2014-06-11|    OWNER|
|1933-11-19|     318480|           60|       5308.0|          D|             0|  POPLATEK MESICNE|1905-12-20|    OWNER|
|1932-04-03|     110736|           48|  

In [0]:
df.printSchema()

root
 |-- date_loan: date (nullable = true)
 |-- loan_amount: integer (nullable = true)
 |-- loan_duration: integer (nullable = true)
 |-- loan_payments: double (nullable = true)
 |-- loan_status: string (nullable = true)
 |-- buen_candidato: integer (nullable = true)
 |-- account_frequency: string (nullable = true)
 |-- date_ac: date (nullable = true)
 |-- type: string (nullable = true)



In [0]:
from pyspark.sql.functions import expr

# Lista de columnas numéricas donde eliminar valores atípicos
numeric_cols = ["loan_amount", "loan_duration", "loan_payments"]

for col_name in numeric_cols:
    # Cálculo de Q1 y Q3
    quantiles = df.approxQuantile(col_name, [0.25, 0.75], 0.05)
    Q1, Q3 = quantiles
    IQR = Q3 - Q1

    # Límites inferior y superior
    lower_bound = Q1 - 1.5 * IQR
    upper_bound = Q3 + 1.5 * IQR

    # Filtrar valores dentro de los límites
    df = df.filter((col(col_name) >= lower_bound) & (col(col_name) <= upper_bound))

df.show()



+----------+-----------+-------------+-------------+-----------+--------------+------------------+----------+---------+
| date_loan|loan_amount|loan_duration|loan_payments|loan_status|buen_candidato| account_frequency|   date_ac|     type|
+----------+-----------+-------------+-------------+-----------+--------------+------------------+----------+---------+
|1957-12-24|      80952|           24|       3373.0|          A|             1|  POPLATEK MESICNE|1930-12-07|    OWNER|
|1957-12-24|      80952|           24|       3373.0|          A|             1|  POPLATEK MESICNE|1930-12-07|DISPONENT|
|2013-08-16|      30276|           12|       2523.0|          B|             0|  POPLATEK MESICNE|1986-03-09|    OWNER|
|1907-01-14|      30276|           12|       2523.0|          A|             1|  POPLATEK MESICNE|2014-06-11|    OWNER|
|1933-11-19|     318480|           60|       5308.0|          D|             0|  POPLATEK MESICNE|1905-12-20|    OWNER|
|1932-04-03|     110736|           48|  

In [0]:
#cantidad de registros y columnas
print(f"Filas: {df.count()} | Columnas: {len(df.columns)}")

Filas: 792 | Columnas: 9


In [0]:
display(df)

date_loan,loan_amount,loan_duration,loan_payments,loan_status,buen_candidato,account_frequency,date_ac,type
1957-12-24,80952,24,3373.0,A,1,POPLATEK MESICNE,1930-12-07,OWNER
1957-12-24,80952,24,3373.0,A,1,POPLATEK MESICNE,1930-12-07,DISPONENT
2013-08-16,30276,12,2523.0,B,0,POPLATEK MESICNE,1986-03-09,OWNER
1907-01-14,30276,12,2523.0,A,1,POPLATEK MESICNE,2014-06-11,OWNER
1933-11-19,318480,60,5308.0,D,0,POPLATEK MESICNE,1905-12-20,OWNER
1932-04-03,110736,48,2307.0,C,1,POPLATEK TYDNE,1905-12-10,OWNER
2013-10-28,165960,24,6915.0,A,1,POPLATEK MESICNE,1960-06-25,OWNER
1905-12-12,102876,12,8573.0,A,1,POPLATEK MESICNE,2013-10-31,OWNER
1905-12-12,102876,12,8573.0,A,1,POPLATEK MESICNE,2013-10-31,DISPONENT
1907-01-12,265320,36,7370.0,D,0,POPLATEK MESICNE,2013-04-19,OWNER


In [0]:
df.createOrReplaceTempView("df")


## Indexar tablas de df (candidatos_creditos)

In [0]:
%sql

-- Crear la tabla minable con indexación para variables categóricas
CREATE OR REPLACE TEMP VIEW tabla_indexada AS
SELECT *,
       -- Indexar 'loan_status' (esto puede ser de acuerdo a tus categorías)
       CASE WHEN loan_status = 'A' THEN 0
            WHEN loan_status = 'B' THEN 1
            WHEN loan_status = 'C' THEN 2
            ELSE 3 END AS loan_status_index,

       -- Indexar 'type' (puedes hacer más reglas según tu dataset)
       CASE WHEN type = 'DISPONENT' THEN 1
            WHEN type = 'OWNER' THEN 2
            ELSE 0 END AS type_index,

       -- Indexar 'account_frequency' si es necesario
       CASE WHEN account_frequency = 'MONTHLY' THEN 0
            WHEN account_frequency = 'AFTER' THEN 1
            ELSE 2 END AS account_frequency_index

FROM df;


## One-Hot Encoding

In [0]:
%sql
-- Crear las columnas One-Hot Encoding
CREATE OR REPLACE TEMP VIEW tabla_onehot AS
SELECT *,
       CASE WHEN loan_status = 'A' THEN 1 ELSE 0 END AS status_A,
       CASE WHEN loan_status = 'B' THEN 1 ELSE 0 END AS status_B,
       CASE WHEN loan_status = 'C' THEN 1 ELSE 0 END AS status_C,
       CASE WHEN loan_status = 'D' THEN 1 ELSE 0 END AS status_D,
       
       CASE WHEN type = 'DISPONENT' THEN 1 ELSE 0 END AS type_DISPONENT,
       CASE WHEN type = 'OWNER' THEN 1 ELSE 0 END AS type_OWNER,
       
       CASE WHEN account_frequency = 'POPLATEK MESICNE' THEN 1 ELSE 0 END AS frequency_POPLATEK_MESICNE,
       CASE WHEN account_frequency = 'POPLATEK PO OBRATU' THEN 1 ELSE 0 END AS frequency_POPLATEK_PO_OBRATU,
       CASE WHEN account_frequency = 'POPLATEK TYDNE' THEN 1 ELSE 0 END AS frequency_POPLATEK_TYDNE

       
FROM tabla_indexada;


In [0]:
%sql
SELECT * FROM tabla_onehot LIMIT 10;


date_loan,loan_amount,loan_duration,loan_payments,loan_status,buen_candidato,account_frequency,date_ac,type,loan_status_index,type_index,account_frequency_index,status_A,status_B,status_C,status_D,type_DISPONENT,type_OWNER,frequency_POPLATEK_MESICNE,frequency_POPLATEK_PO_OBRATU,frequency_POPLATEK_TYDNE
1957-12-24,80952,24,3373.0,A,1,POPLATEK MESICNE,1930-12-07,OWNER,0,2,2,1,0,0,0,0,1,1,0,0
1957-12-24,80952,24,3373.0,A,1,POPLATEK MESICNE,1930-12-07,DISPONENT,0,1,2,1,0,0,0,1,0,1,0,0
2013-08-16,30276,12,2523.0,B,0,POPLATEK MESICNE,1986-03-09,OWNER,1,2,2,0,1,0,0,0,1,1,0,0
1907-01-14,30276,12,2523.0,A,1,POPLATEK MESICNE,2014-06-11,OWNER,0,2,2,1,0,0,0,0,1,1,0,0
1933-11-19,318480,60,5308.0,D,0,POPLATEK MESICNE,1905-12-20,OWNER,3,2,2,0,0,0,1,0,1,1,0,0
1932-04-03,110736,48,2307.0,C,1,POPLATEK TYDNE,1905-12-10,OWNER,2,2,2,0,0,1,0,0,1,0,0,1
2013-10-28,165960,24,6915.0,A,1,POPLATEK MESICNE,1960-06-25,OWNER,0,2,2,1,0,0,0,0,1,1,0,0
1905-12-12,102876,12,8573.0,A,1,POPLATEK MESICNE,2013-10-31,OWNER,0,2,2,1,0,0,0,0,1,1,0,0
1905-12-12,102876,12,8573.0,A,1,POPLATEK MESICNE,2013-10-31,DISPONENT,0,1,2,1,0,0,0,1,0,1,0,0
1907-01-12,265320,36,7370.0,D,0,POPLATEK MESICNE,2013-04-19,OWNER,3,2,2,0,0,0,1,0,1,1,0,0


## Creacion de la tabla minable

In [0]:
%sql
-- Crear la tabla definitiva en la base de datos minable con la columna 'buen_candidato' 
CREATE OR REPLACE TABLE tabla_minable AS
SELECT date_loan,
       loan_amount, 
       loan_duration,
       loan_payments,
       status_A, 
       status_B, 
       status_C, 
       status_D,
       type_DISPONENT, 
       type_OWNER,
       frequency_POPLATEK_MESICNE, 
       frequency_POPLATEK_PO_OBRATU, 
       frequency_POPLATEK_TYDNE,
       date_ac,
       buen_candidato
FROM tabla_onehot;


num_affected_rows,num_inserted_rows


In [0]:
%sql
DESCRIBE tabla_onehot;



col_name,data_type,comment
date_loan,date,null
loan_amount,int,null
loan_duration,int,null
loan_payments,double,null
loan_status,string,null
buen_candidato,int,null
account_frequency,string,null
date_ac,date,null
type,string,null
loan_status_index,int,null


In [0]:
%sql
-- Consulta simple para visualizar las primeras filas de la tabla
SELECT * FROM tabla_onehot LIMIT 100;


date_loan,loan_amount,loan_duration,loan_payments,loan_status,buen_candidato,account_frequency,date_ac,type,loan_status_index,type_index,account_frequency_index,status_A,status_B,status_C,status_D,type_DISPONENT,type_OWNER,frequency_POPLATEK_MESICNE,frequency_POPLATEK_PO_OBRATU,frequency_POPLATEK_TYDNE
1957-12-24,80952,24,3373.0,A,1,POPLATEK MESICNE,1930-12-07,OWNER,0,2,2,1,0,0,0,0,1,1,0,0
1957-12-24,80952,24,3373.0,A,1,POPLATEK MESICNE,1930-12-07,DISPONENT,0,1,2,1,0,0,0,1,0,1,0,0
2013-08-16,30276,12,2523.0,B,0,POPLATEK MESICNE,1986-03-09,OWNER,1,2,2,0,1,0,0,0,1,1,0,0
1907-01-14,30276,12,2523.0,A,1,POPLATEK MESICNE,2014-06-11,OWNER,0,2,2,1,0,0,0,0,1,1,0,0
1933-11-19,318480,60,5308.0,D,0,POPLATEK MESICNE,1905-12-20,OWNER,3,2,2,0,0,0,1,0,1,1,0,0
1932-04-03,110736,48,2307.0,C,1,POPLATEK TYDNE,1905-12-10,OWNER,2,2,2,0,0,1,0,0,1,0,0,1
2013-10-28,165960,24,6915.0,A,1,POPLATEK MESICNE,1960-06-25,OWNER,0,2,2,1,0,0,0,0,1,1,0,0
1905-12-12,102876,12,8573.0,A,1,POPLATEK MESICNE,2013-10-31,OWNER,0,2,2,1,0,0,0,0,1,1,0,0
1905-12-12,102876,12,8573.0,A,1,POPLATEK MESICNE,2013-10-31,DISPONENT,0,1,2,1,0,0,0,1,0,1,0,0
1907-01-12,265320,36,7370.0,D,0,POPLATEK MESICNE,2013-04-19,OWNER,3,2,2,0,0,0,1,0,1,1,0,0


In [0]:
df = spark.sql("SELECT * FROM default.tabla_minable")
display(df)

date_loan,loan_amount,loan_duration,loan_payments,status_A,status_B,status_C,status_D,type_DISPONENT,type_OWNER,frequency_POPLATEK_MESICNE,frequency_POPLATEK_PO_OBRATU,frequency_POPLATEK_TYDNE,date_ac,buen_candidato
1957-12-24,80952,24,3373.0,1,0,0,0,0,1,1,0,0,1930-12-07,1
1957-12-24,80952,24,3373.0,1,0,0,0,1,0,1,0,0,1930-12-07,1
2013-08-16,30276,12,2523.0,0,1,0,0,0,1,1,0,0,1986-03-09,0
1907-01-14,30276,12,2523.0,1,0,0,0,0,1,1,0,0,2014-06-11,1
1933-11-19,318480,60,5308.0,0,0,0,1,0,1,1,0,0,1905-12-20,0
1932-04-03,110736,48,2307.0,0,0,1,0,0,1,0,0,1,1905-12-10,1
2013-10-28,165960,24,6915.0,1,0,0,0,0,1,1,0,0,1960-06-25,1
1905-12-12,102876,12,8573.0,1,0,0,0,0,1,1,0,0,2013-10-31,1
1905-12-12,102876,12,8573.0,1,0,0,0,1,0,1,0,0,2013-10-31,1
1907-01-12,265320,36,7370.0,0,0,0,1,0,1,1,0,0,2013-04-19,0


In [0]:
%sql
SELECT DISTINCT account_frequency FROM df;


account_frequency
POPLATEK MESICNE
POPLATEK TYDNE
POPLATEK PO OBRATU


## Estandarizacion

In [0]:
from pyspark.ml.feature import VectorAssembler, StandardScaler

# Columnas numéricas a estandarizar
numerical_cols = ["loan_amount", "loan_duration", "loan_payments"]

# Combinar las columnas numéricas en un vector
assembler = VectorAssembler(inputCols=numerical_cols, outputCol="features_vector")
df_vector = assembler.transform(df)

# Aplicar StandardScaler para estandarizar
scaler = StandardScaler(inputCol="features_vector", outputCol="scaled_features", withMean=True, withStd=True)
scaler_model = scaler.fit(df_vector)
df_scaled = scaler_model.transform(df_vector)

# Separar los valores escalados de regreso a las columnas originales
from pyspark.ml.functions import vector_to_array
df_scaled = df_scaled.withColumn("scaled_array", vector_to_array("scaled_features"))

# Reemplazar las columnas originales con las escaladas
for i, col_name in enumerate(numerical_cols):
    df_scaled = df_scaled.withColumn(col_name, df_scaled["scaled_array"][i])

# Eliminar columnas temporales
df_final = df_scaled.drop("features_vector", "scaled_features", "scaled_array")

# Mostrar el DataFrame final con las columnas originales sobrescritas
df_final.show()


+----------+--------------------+-------------------+--------------------+--------+--------+--------+--------+--------------+----------+--------------------------+----------------------------+------------------------+----------+--------------+
| date_loan|         loan_amount|      loan_duration|       loan_payments|status_A|status_B|status_C|status_D|type_DISPONENT|type_OWNER|frequency_POPLATEK_MESICNE|frequency_POPLATEK_PO_OBRATU|frequency_POPLATEK_TYDNE|   date_ac|buen_candidato|
+----------+--------------------+-------------------+--------------------+--------+--------+--------+--------+--------------+----------+--------------------------+----------------------------+------------------------+----------+--------------+
|1957-12-24| -0.5989522526838829|-0.6775296006865399|-0.32128816469640664|       1|       0|       0|       0|             0|         1|                         1|                           0|                       0|1930-12-07|             1|
|1957-12-24| -0.59895225

In [0]:
display(df_final)

date_loan,loan_amount,loan_duration,loan_payments,status_A,status_B,status_C,status_D,type_DISPONENT,type_OWNER,frequency_POPLATEK_MESICNE,frequency_POPLATEK_PO_OBRATU,frequency_POPLATEK_TYDNE,date_ac,buen_candidato
1957-12-24,-0.5989522526838829,-0.6775296006865399,-0.32128816469640664,1,0,0,0,0,1,1,0,0,1930-12-07,1
1957-12-24,-0.5989522526838829,-0.6775296006865399,-0.32128816469640664,1,0,0,0,1,0,1,0,0,1930-12-07,1
2013-08-16,-1.1234700933711146,-1.3835867635072503,-0.7294779447662366,0,1,0,0,0,1,1,0,0,1986-03-09,0
1907-01-14,-1.1234700933711146,-1.3835867635072503,-0.7294779447662366,1,0,0,0,0,1,1,0,0,2014-06-11,1
1933-11-19,1.8595621064359527,1.4406418877755913,0.6079438640507945,0,0,0,1,0,1,1,0,0,1905-12-20,0
1932-04-03,-0.2906753688132437,0.7345847249548809,-0.8332061712310405,0,0,1,0,0,1,0,0,1,1905-12-10,1
2013-10-28,0.280916178153993,-0.6775296006865399,1.3796626600181083,1,0,0,0,0,1,1,0,0,1960-06-25,1
1905-12-12,-0.3720296633077113,-1.3835867635072503,2.175872842789612,1,0,0,0,0,1,1,0,0,2013-10-31,1
1905-12-12,-0.3720296633077113,-1.3835867635072503,2.175872842789612,1,0,0,0,1,0,1,0,0,2013-10-31,1
1907-01-12,1.3093338245878738,0.02852756213417051,1.5981642481731348,0,0,0,1,0,1,1,0,0,2013-04-19,0


In [0]:
#display(df_final.write.csv("/Users/201602892@est.umss.edu/tabla_minable_estandarizada.csv", header=True))
